In [8]:
from dask.distributed import Client
import numpy as np
import dask.dataframe as dd
import ast
import re

In [2]:
client = Client('127.0.0.1:8786')
client = client.restart()  

In [3]:
reviews = dd.read_csv('data/user_reviews.csv')
products = dd.read_csv('data/products.csv')

In [4]:
temp = reviews[['reviewerID', 'asin', 'helpful', 'overall', 'reviewTime']]

In [ ]:
temp['helpful_votes'] = temp['helpful'].apply(lambda x: int(x[1:-1].split(',')[0]), meta=('helpful', 'object'))
temp['total_votes'] = temp['helpful'].apply(lambda x: int(x[1:-1].split(',')[1]), meta=('helpful', 'object'))
temp.helpful_votes = temp.helpful_votes.astype(np.int16)
temp.total_votes = temp.total_votes.astype(np.int16)
temp = temp.drop(columns='helpful')

In [6]:
temp['reviewTime'] = temp['reviewTime'].apply(lambda x: int(x[-4:]), meta=('reviewTime', 'object'))
temp.reviewTime = temp.reviewTime.astype(np.int16)

In [8]:
times = temp.groupby('reviewerID').agg({'overall': 'mean', 'asin': 'count', 'helpful_votes': 'sum', 'total_votes':'sum', 'reviewTime': 'min'}, split_out=10).persist()

In [9]:
times.describe().compute()

,overall,asin,helpful_votes,total_votes,reviewTime
count,1.367709e+07,1.367709e+07,1.367709e+07,1.367709e+07,1.367709e+07
mean,4.097750e+00,2.720221e+00,5.619712e+00,7.582020e+00,2.011140e+03
std,1.213044e+00,1.072664e+01,7.188327e+01,8.387787e+01,3.326688e+00
min,1.000000e+00,1.000000e+00,-2.799500e+04,-2.752900e+04,1.996000e+03
25%,3.666667e+00,1.000000e+00,0.000000e+00,0.000000e+00,2.010000e+03
50%,4.666667e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.012000e+03
75%,5.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00,2.013000e+03
max,5.000000e+00,2.009100e+04,9.346100e+04,1.124490e+05,2.014000e+03


In [82]:
q4 = products[['categories', 'asin']].dropna()

def categorize(x):
    lst = eval(x)
    try:
        return lst[0][0]
    except:
        return ''
q4['categories'] = q4['categories'].apply(categorize, meta= 'string')

In [34]:
q1a = reviews.isna().mean().round(4).compute() * 100
q1b = products.isna().mean().round(4).compute() * 100

In [35]:
df_merged = reviews[['asin', 'overall']].merge(products[['asin', 'price']], on='asin')
q2 = df_merged[['price', 'overall']].corr().compute()

In [36]:
q3 = products.price.describe().compute()

In [83]:
q4 = q4.groupby('categories').count().compute()

In [48]:
q5 = int(len(df_merged) != len(reviews))

In [15]:
q5

1

In [7]:
products.related.compute().values[:10]

array([nan,
       "{'also_viewed': ['B0036FO6SI', 'B000KL8ODE', '000014357X', 'B0037718RC', 'B002I5GNVU', 'B000RBU4BM'], 'buy_after_viewing': ['B0036FO6SI', 'B000KL8ODE', '000014357X', 'B0037718RC']}",
       "{'also_viewed': ['B00JO8II76', 'B00DGN4R1Q', 'B00E1YRI4C']}",
       "{'also_bought': ['0000032050', 'B00D0DJAEG', '0000032042', 'B00D0F450I', 'B00D2JTMS2', 'B00D0FDUAY', 'B00D2JSRFQ', '0000032034', 'B00D0D5F6S', 'B00D2JRWWA', 'B00D0FIIJM', 'B00D0FCQQI', 'B00EXVN9PU', 'B0041EOTJO', 'B004PYEE8G', 'B001GTKPDQ', 'B00EON0SJ2', 'B005HMHOQ4', 'B002XZMGGQ'], 'also_viewed': ['B00D0F450I', '0000032050', 'B00D2JTMS2', '0000032042', 'B004PYEE8G', 'B00JHNSNSM', 'B00D0DJAEG', 'B00D2JSRFQ', 'B00D0FCQQI', 'B00D2JRWWA', 'B003AVNY6I', 'B0071KR2LC', 'B00GOR07RE', 'B00D0FIIJM', 'B005F50FXC', 'B0079MCIMU', 'B00D0FDUAY', 'B00H3RYN3I', 'B005C4Y4F6', 'B007IEFT84', 'B00D0D5F6S', 'B002BZX8Z6', 'B00JHONN1S', 'B008F0SU0Y', 'B00FNNFXAG', 'B007R2RM8W', 'B007VM3AMK', 'B00C0PLENA', 'B00BJGG6VG', 'B00E1YRI4C',

In [30]:
def categorize_2(x):
    try:
        suggestion_dict = eval(x)
        l = suggestion_dict.values()
        return list([item for sublist in l for item in sublist if item != np.nan])
    except:
        return []

def get_ids(dict_string):
    # Get the [id1, id2, id3] string with regex
    return re.findall("[A-Z0-9]{10}", dict_string)

q6 = products[['related']]
q6.columns = ['asin']
q6 = q6['asin'].dropna().apply(get_ids, meta=('asin', 'str'))
q6.explode().drop_duplicates().head(20)
                              ##q6['asin'] = q6['asin'].apply(categorize_2, meta=('asin', 'object'))
#q6.explode('asin').drop_duplicates().compute()
#len(q6.merge(products[['asin']], on='asin').compute()) == len(q6.compute())


#len(q6)

/home/ubuntu/dask_env/lib/python3.6/site-packages/dask/core.py:118: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]
/home/ubuntu/dask_env/lib/python3.6/site-packages/dask/core.py:118: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]
/home/ubuntu/dask_env/lib/python3.6/site-packages/dask/core.py:118: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]


KeyboardInterrupt: 

In [14]:
products.count().compute()

asin           9430088
salesRank      6976727
imUrl          9260789
categories     9354832
title          7997348
description    5496027
price          6063208
related        6691673
brand          1727194
dtype: int64

In [67]:
q4.head()

,asin
categories,
,194914
Books,2369971
CDs & Vinyl,491713
"Clothing, Shoes & Jewelry",1435869
Movies & TV,194874


In [10]:
s = "{'also_viewed': ['B0036FO6SI', 'B000KL8ODE', '000014357X', 'B0037718RC', 'B002I5GNVU', 'B000RBU4BM'], 'buy_after_viewing': ['B0036FO6SI', 'B000KL8ODE', '000014357X', 'B0037718RC']}"

rs = re.findall(r"\b\w{10}\b", s)
rs

['B0036FO6SI',
 'B000KL8ODE',
 '000014357X',
 'B0037718RC',
 'B002I5GNVU',
 'B000RBU4BM',
 'B0036FO6SI',
 'B000KL8ODE',
 '000014357X',
 'B0037718RC']